<a href="https://colab.research.google.com/github/life-ofpi/dacon_loan/blob/main/model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', family='NanumGothic')
SEED = 42

In [23]:
train=pd.read_csv("/content/drive/MyDrive/dacon/대출등급/open (12)/open/train.csv")
test=pd.read_csv("/content/drive/MyDrive/dacon/대출등급/open (12)/open/train.csv")
sample_submission=pd.read_csv("/content/drive/MyDrive/dacon/대출등급/open (12)/open/sample_submission.csv")
train

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A
3,TRAIN_00003,14400000,36 months,8 years,MORTGAGE,132000000,15.09,15,부채 통합,0,325824,153108.0,0.0,0.0,C
4,TRAIN_00004,18000000,60 months,Unknown,RENT,71736000,25.39,19,주요 구매,0,228540,148956.0,0.0,0.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96289,TRAIN_96289,14400000,36 months,10+ years,MORTGAGE,210000000,9.33,33,신용 카드,0,974580,492168.0,0.0,0.0,C
96290,TRAIN_96290,28800000,60 months,10+ years,MORTGAGE,132000000,5.16,25,주택 개선,0,583728,855084.0,0.0,0.0,E
96291,TRAIN_96291,14400000,36 months,1 year,MORTGAGE,84000000,11.24,22,신용 카드,0,1489128,241236.0,0.0,0.0,A
96292,TRAIN_96292,15600000,36 months,5 years,MORTGAGE,66330000,17.30,21,부채 통합,2,1378368,818076.0,0.0,0.0,D


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
train.isnull().sum(axis=0)
#결측치x

ID              0
대출금액            0
대출기간            0
근로기간            0
주택소유상태          0
연간소득            0
부채_대비_소득_비율     0
총계좌수            0
대출목적            0
최근_2년간_연체_횟수    0
총상환원금           0
총상환이자           0
총연체금액           0
연체계좌수           0
대출등급            0
dtype: int64

이산형 변수
-ID, 대출기간, 주택소유상태, 대출목적, 대출등급, 근로기간
연속형 변수
-대출금액, 연간소득, 부채대비 소득비율, 총계좌수,최근 2년간 연체횟수, 총상환원금, 총상환이자, 총연체금액, 연체계좌수


In [24]:
from sklearn.preprocessing import LabelEncoder

# '근로기간'을 처리하는 함수
def convert_gunro(x):
    try:
        if 'year' in x:
            # '10+'인 경우에는 10으로 반환하고, 그 외에는 정수로 변환
            return 10 if '10+' in x else int(x.split()[0])
        elif '+' in x:
            return int(x.split('+')[0])
        else:
            return 0
    except ValueError:
        # 변환 중에 오류가 발생하면 기본값인 0을 반환합니다.
        return 0

# 데이터 전처리
def preprocess_data(df):
    df['대출기간'] = df['대출기간'].apply(lambda x: int(x.split()[0]))

    # '근로기간'을 처리
    df['근로기간'] = df['근로기간'].apply(convert_gunro)

    # '주택소유상태'를 Label Encoding으로 변환
    label_encoder = LabelEncoder()
    df['주택소유상태'] = label_encoder.fit_transform(df['주택소유상태'])
    df['대출목적'] = label_encoder.fit_transform(df['대출목적'])

    return df

# 데이터 전처리 함수 적용
train = preprocess_data(train)
test = preprocess_data(test)



In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV


In [26]:
# 특성과 타겟 변수 정의
X = train.drop(['ID', '대출등급'], axis=1)
y = train['대출등급']

# 훈련 및 검증 세트로 데이터 나누기
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Classifier 모델 정의
rf_model = RandomForestClassifier(random_state=42)

# 모델 학습
rf_model.fit(X_train, y_train)

# 검증 세트에서 예측 수행
predictions = rf_model.predict(X_valid)

# 모델 평가
accuracy = accuracy_score(y_valid, predictions)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.80


In [27]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 최적의 모델을 이용하여 예측
best_rf_model = grid_search.best_estimator_
best_rf_predictions = best_rf_model.predict(X_valid)

# 최적의 모델 평가
best_accuracy = accuracy_score(y_valid, best_rf_predictions)
print(f"Best Model Accuracy: {best_accuracy:.2f}")


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best Model Accuracy: 0.80


In [37]:
# 학습 시 사용된 특성의 이름 확인
feature_names_train = X.columns.tolist()

# 테스트 데이터의 특성을 일치시키기
test_data_for_prediction = test[feature_names_train]

# 테스트 데이터에 대한 예측 수행
test_predictions = best_rf_model.predict(test_data_for_prediction)

# 예측 결과를 DataFrame으로 만들기
submission_df = pd.DataFrame({'ID': test['ID'], '대출등급': test_predictions})

# 결과를 CSV 파일로 저장
submission_df.to_csv('/content/drive/MyDrive/dacon/대출등급/open (12)/open/submission.csv', index=False)
